In [96]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import glob
import pandas as pd
import tensorflow as tf
from PIL import Image


In [185]:
def imageToMatrix (filepath):
    images = glob.glob(filepath)
    X_list = []
    
    #looping through images in folder
    for myFile in images:
      image = cv.imread (myFile)
      #feeding in the images resized as 100X100 to reduce the size of the training matrix
      resized = cv.resize(image, (100,100), interpolation = cv.INTER_AREA)
      #code below changes to image to gray, reducing image color channel to one
      color_image = cv.cvtColor(resized, cv.COLOR_BGR2RGB)
      gray_image = cv.cvtColor(color_image, cv.COLOR_RGB2GRAY)
      X_list.append (gray_image)
        
    #Transforming list of pixel values into an numpy matrix
    
    X_data = np.asarray(X_list)
    
    #normalizing pixel values and converting dtype to value accepted by tensorflow model 
    X_data = np.array(X_data).astype(np.float32)/255
    
    return X_data

In [178]:
def jpgtoMatrix (filename):
#     image = cv.imread(UPLOAD_FOLDER+'/'+filename)
    image = cv.imread(filename)
    X_list = []
    resized = cv.resize(image, (100,100), interpolation = cv.INTER_AREA)
    color_image = cv.cvtColor(resized, cv.COLOR_BGR2RGB)
    gray_image = cv.cvtColor(color_image, cv.COLOR_RGB2GRAY)
    X_list.append (gray_image)  
    X_data = np.asarray(X_list)
    X_data = np.array(X_data).astype(np.float32)/255
    
    return X_data

In [179]:
#since I would like to deploy my app to Google App Engine, which doesn't support cv library, need to create a function 
#using PIL that will convert images to numpy pixel array that can be fed into TF model. Th

def pilMatrix(filepath):

    image = Image.open(filepath).convert('L')
    
    X_list = []
    resized = image.resize((100,100), Image.ANTIALIAS)
    #need to perform this step so np array is same shape as ML model expects. 
    resized = np.asarray(resized)
    X_list.append (resized)  
    X_data = np.asarray(X_list)
    X_data = np.array(X_data).astype(np.float32)/255

    
    return X_data

In [183]:
#getting training data for hotdog images
train_hotdog_dataset = imageToMatrix('./seefood/train/hot_dog/*.jpg')

In [184]:
train_hotdog_dataset.shape

(249, 100, 100)

In [5]:
#creating numpy array of labels we can add to final numpy training matrix
train_labels_hotdog = np.repeat(1, 249)

#converting labels to int32 for TF model
train_labels_hotdog = np.array(train_labels_hotdog).astype(np.int32)

In [6]:
train_labels_hotdog.shape

(249,)

In [7]:
#Getting training data for not hotdog image
train_not_hotdog_dataset = imageToMatrix('./seefood/train/not_hot_dog/*.jpg')

train_not_hotdog_lables = np.repeat(0, 249)
train_not_hotdog_lables = np.array(train_not_hotdog_lables).astype(np.int32)

train_not_hotdog_dataset.shape


(249, 100, 100)

In [8]:
#Concatenating arrays together to create training data array and 
#training label array

train_data = np.concatenate([train_hotdog_dataset, train_not_hotdog_dataset])
train_labels = np.concatenate([train_labels_hotdog, train_not_hotdog_lables])


In [9]:
# Getting index of train data and shuffling it. 
# In Numpy we can index arrays based on index of other arrays. If we 
# shuffle index of the train data, we can apply that to our labels 
# array and both train and label arrays will match up with each other
s = np.arange(train_data.shape[0])
np.random.shuffle(s)

In [10]:
# Applying shuffled index to both train and train labels array
train_data = train_data[s]
train_labels = train_labels[s]


In [11]:
#Now getting test data, note test data has 250 images instead of 249 

test_hotdog_data = imageToMatrix('./seefood/test/hot_dog/*.jpg')

test_hotdog_labels = np.repeat(1, 250)
test_hotdog_labels= np.array(test_hotdog_labels).astype(np.int32)

test_not_hotdog_data = imageToMatrix('./seefood/test/not_hot_dog/*.jpg')

test_not_hotdog_labels = np.repeat(0, 250)
test_not_hotdog_labels= np.array(test_not_hotdog_labels).astype(np.int32)

#combining two arrays together

test_data = np.concatenate([test_hotdog_data, test_not_hotdog_data])
test_labels = np.concatenate([test_hotdog_labels, test_not_hotdog_labels])

#shuffling data

s = np.arange(test_data.shape[0])
np.random.shuffle(s)

test_data = test_data[s]
test_labels = test_labels[s]


In [16]:
#Now the fun stuff! Getting to create our CNN input function for our Tensorflow estimator

def cnn_model(features, labels, mode):
    #input layer, -1 is batch size treated as hyperparmeter, 100X100 is size of image, 1 is number of color channels
    input_layer = tf.reshape(features["x"], [-1, 100, 100, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5,5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    #convolutional and pooling layer#3 
    conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    #convolutional and pooling layer #4 
   
 # Dense Layer
    pool4_flat = tf.reshape(pool3, [-1, 12 * 12 * 64])
    
    dense = tf.layers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Hotdog output Layer
    hotdogs = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=hotdogs, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(hotdogs, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=hotdogs)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [17]:
# Create the Estimator, specifying input function and where directory 
# model will be written to
hotdog_classifier = tf.estimator.Estimator(
    model_fn=cnn_model, model_dir="./cnn_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb3a987d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [19]:
# Input function for training the model 
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
hotdog_classifier.train(
    input_fn=train_input_fn,
    steps=1, 
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-1
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 1 into ./cnn_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.48896503 0.51103497]
 [0.45905164 0.54094833]
 [0.4731237  0.5268763 ]
 [0.48483884 0.51516116]
 [0.4884452  0.51155484]
 [0.46043143 0.5395686 ]
 [0.46497652 0.5350235 ]
 [0.50068367 0.49931636]
 [0.43304    0.56696   ]
 [0.48608914 0.51391083]]
INFO:tensorflow:loss = 0.7269672, step = 2
INFO:tensorflow:Saving checkpoints for 2 into ./cnn_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.7269672.


In [20]:
#training step, note we are training locally so only using 2000 epochs
hotdog_classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-2
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./cnn_model/model.ckpt.
INFO:tensorflow:loss = 0.6803946, step = 3
INFO:tensorflow:global_step/sec: 3.43157
INFO:tensorflow:loss = 0.67617357, step = 103 (29.142 sec)
INFO:tensorflow:global_step/sec: 3.04756
INFO:tensorflow:loss = 0.6829586, step = 203 (32.813 sec)
INFO:tensorflow:global_step/sec: 3.41215
INFO:tensorflow:loss = 0.67568696, step = 303 (29.312 sec)
INFO:tensorflow:global_step/sec: 2.66268
INFO:tensorflow:loss = 0.7082362, step = 403 (37.551 sec)
INFO:tensorflow:global_step/sec: 3.21487
INFO:tensorflow:loss = 0.68916714, step = 503 (31.106 sec)
INFO:tensorflow:global_step/sec: 3.20758
INFO:tensorflow:loss = 0.6584821, step = 603

In [21]:
#Evaluating Model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": test_data},
    y = test_labels,
    num_epochs = 1,
    shuffle = False)

eval_results = hotdog_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-30T03:42:54Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-30-03:43:00
INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.582, global_step = 2002, loss = 0.6802262
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2002: ./cnn_model/model.ckpt-2002
{'accuracy': 0.582, 'loss': 0.6802262, 'global_step': 2002}


In [46]:
#Now lets try feeding our model an image of a hotdog and see how it performs
hotdog_image = imageToMatrix('./yummyhotdog.jpg')


In [47]:
#Creating a prediction input function for our model. Class = 1 is hotdog, class=2 is not hotdog
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": hotdog_image},
    y =None,
    num_epochs = 1,
    shuffle = False)

predict = list(hotdog_classifier.predict(predict_input_fn))

predict[0]


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'classes': 1, 'probabilities': array([0.3631544, 0.6368456], dtype=float32)}

In [48]:
#Now lets test our model with a not hotdog image from training data 
not_hotdog = imageToMatrix('./seefood/train/not_hot_dog/6127.jpg')

In [49]:
nothotdoginput = tf.estimator.inputs.numpy_input_fn(
    x = {"x": not_hotdog},
    y =None,
    num_epochs = 1,
    shuffle = False)

In [50]:
predict = list(hotdog_classifier.predict(input_fn=nothotdoginput))

predict[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'classes': 0, 'probabilities': array([0.65914315, 0.34085688], dtype=float32)}

In [51]:
#exporting the TF estimator model so we can host model on Google Cloud AI Platform. We start by defining a input 
#reciever function then define directory we would like to save model to

def serving_input_rec_fn():
    #inputs should describe the type of input we are going to serve to model, in this case a numpy array of shape (100, 100, 1)
    inputs = {
        "x": tf.placeholder(tf.float32, [None, 100, 100, 1]), 
    }
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

hotdog_classifier.export_savedmodel("./saved_model/", serving_input_rec_fn,
                            strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-2002
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/temp-b'1562014990'/saved_model.pb


b'./saved_model/1562014990'